In [1]:
from datetime import datetime

from setuptools.sandbox import save_argv

from tool import preprocess
from tool import customdataset

In [2]:
import pandas as pd
from collections import Counter
import torch
from sklearn.model_selection import train_test_split
from networkx.readwrite.json_graph import adjacency
import faiss


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
path = 'D:\\VideoRecSystem\\MicroLens\\DataSet\\MicroLens-50k_pairs.csv'
user = 'user'
item = 'item'
user_id = 'user_id'
item_id = 'item_id'
timestamp = 'timestamp'
save_dir = './embeddings'
top_k = 10
# path = pd.read_csv('MicroLens-50k_pairs.csv')

In [5]:
dataset_pd,num_users,num_items = preprocess.openAndSort(path,user_id=user,item_id=item,timestamp='timestamp')

dataset base information：
- number of users：50000
- number of items：19220
- number of rows：359708


In [6]:
# # order by user,timestamp
# filtered_df = dataset_pd.sort_values(by=["user", "timestamp"])


In [7]:
# def split(df, user_id, item_id, timestamp):
#
#     # 获取每个用户的最后一条记录作为 test
#     test_df = df.groupby(user_id).tail(1)
#     train_df = df.drop(index=test_df.index)
#
#     # 过滤 test 中那些 user/item 不在 train 中的
#     train_users = set(train_df[user_id])
#     train_items = set(train_df[item_id])
#
#     # 确保测试集中出现的用户/物品都在训练集中出现过，避免某个物品仅出现在测试集中，没有在训练集中得到过训练
#     test_df = test_df[
#         test_df[user_id].isin(train_users) &
#         test_df[item_id].isin(train_items)
#     ]
#     # .reset_index重置 df 的索引，使得不连续的索引重新排列整齐，drop=True表明旧的索引不再保留
#     return train_df.reset_index(drop=True), test_df.reset_index(drop=True)

In [8]:

train_df, test_df = preprocess.split(dataset_pd,user, item, timestamp)
print(f"Train size: {len(train_df)}")
print(f"Test size: {len(test_df)}")


Train size: 309708
Test size: 49424


In [9]:
# maintain a map from new id to old id, new id for constructing matrix
user2id = {u: i for i, u in enumerate(dataset_pd[user].unique())}
item2id = {i: j for j, i in enumerate(dataset_pd[item].unique())}

# apply to train_df and test_df
train_df[user_id] = train_df[user].map(user2id)
train_df[item_id] = train_df[item].map(item2id)
test_df[user_id] = test_df[user].map(user2id)
test_df[item_id] = test_df[item].map(item2id)



In [10]:
# DSSM implementation in PyTorch

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random


class DSSM(nn.Module):
    """
    双塔 DSSM：用户塔 + 物品塔
    hidden_dims 如 [128, 64]，最后输出维度 = hidden_dims[-1]
    """
    def __init__(self, num_users, num_items, emb_dim=64, hidden_dims=(128, 64), dropout=0.2):
        super().__init__()
        self.user_emb = nn.Embedding(num_users, emb_dim)
        self.item_emb = nn.Embedding(num_items, emb_dim)

        # 🔸 初始化（可选但推荐）
        nn.init.xavier_uniform_(self.user_emb.weight)
        nn.init.xavier_uniform_(self.item_emb.weight)

        def _build_mlp(in_dim, hidden_dims):
            layers = []
            for h in hidden_dims:
                layers += [nn.Linear(in_dim, h),
                           nn.ReLU(),
                           nn.Dropout(dropout)]
                in_dim = h
            return nn.Sequential(*layers)

        self.mlp_user = _build_mlp(emb_dim, hidden_dims)
        self.mlp_item = _build_mlp(emb_dim, hidden_dims)

    def forward(self, user_id, item_id, l2_norm=True):
        """
        返回:
          score: (B,) 点积分数
          u_vec, i_vec: (B, d) 两侧向量
        """
        u = self.user_emb(user_id)          # (B, emb_dim)
        i = self.item_emb(item_id)          # (B, emb_dim)

        u_vec = self.mlp_user(u)            # (B, d)
        i_vec = self.mlp_item(i)            # (B, d)

        if l2_norm:
            u_vec = F.normalize(u_vec, p=2, dim=1)
            i_vec = F.normalize(i_vec, p=2, dim=1)

        score = (u_vec * i_vec).sum(dim=1)  # (B,)
        return score, u_vec, i_vec
    def get_users_embedding(self,user_ids,l2_norm=True):
        u = self.user_emb(user_ids)          # (B, emb_dim)

        u_vec = self.mlp_user(u)            # (B, d)

        if l2_norm:
            u_vec = F.normalize(u_vec, p=2, dim=1)
        return u_vec
    def get_items_embedding(self,item_ids,l2_norm=True):
        i = self.item_emb(item_ids)          # (B, emb_dim)

        i_vec = self.mlp_item(i)            # (B, d)

        if l2_norm:
            i_vec = F.normalize(i_vec, p=2, dim=1)
        return i_vec

    def save_embeddings(self, num_users, num_items, device, save_dir='./embeddings'):
        import os
        import faiss
        os.makedirs(save_dir, exist_ok=True)

        self.eval()
        self.to(device)

        user_ids = torch.arange(num_users, dtype=torch.long, device=device)
        item_ids = torch.arange(num_items, dtype=torch.long, device=device)

        with torch.no_grad():
            user_embeds = self.get_users_embedding(user_ids, l2_norm=True)
            item_embeds = self.get_items_embedding(item_ids, l2_norm=True)

        user_embeds = user_embeds.cpu().numpy().astype(np.float32)
        item_embeds = item_embeds.cpu().numpy().astype(np.float32)

        # 保存向量
        np.save(f"{save_dir}/user_embeddings.npy", user_embeds)
        np.save(f"{save_dir}/item_embeddings.npy", item_embeds)

        # 构建 FAISS index（使用内积）
        dim = item_embeds.shape[1]
        index = faiss.IndexFlatIP(dim)
        index.add(item_embeds)

        faiss.write_index(index, f"{save_dir}/item_index.faiss")
        print("Saved user/item embeddings and FAISS index.")



In [11]:
import torch
import numpy as np


def train_model(model,
                train_df,
                num_items,
                epochs=10,
                batch_size=64,
                lr=1e-3,
                print_every=1,
                max_grad_norm=None,
                device=None):

    if device is None:
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    train_data_loader = customdataset.build_train_loader(train_df,num_items,user_id, item_id, batch_size=batch_size ,k_neg=1,num_workers=1)

    for epoch in range(1, epochs + 1):
        model.train()         # train model
        dt_start = datetime.now()
        epoch_loss = 0.0
        for batch in train_data_loader:

            # 1.负采样
            user_ids, pos_ids, neg_ids = batch
            # 2.搬设备
            user_ids = user_ids.to(device)   # 已经是 LongTensor
            pos_ids  = pos_ids.to(device)
            neg_ids  = neg_ids.to(device)

                # -------- 展平负样本 --------
            if neg_ids.ndim == 1:          # 单负样本
                B = neg_ids.size(0)
                u_rep   = user_ids
                neg_flat = neg_ids
                K = 1
            else:                          # 多负样本
                B, K = neg_ids.shape
                u_rep   = user_ids.unsqueeze(1).expand(-1, K).reshape(-1)
                neg_flat = neg_ids.reshape(-1)
                # 把 user 展平重复 K 次，再与展平后的 neg_id 对齐
                u_rep   = user_ids.unsqueeze(1).expand(-1, K).reshape(-1)   # (B*K,)
                neg_flat = neg_ids.reshape(-1)                              # (B*K,)


            # 3.计算得分
            neg_score, _, _ = model(u_rep, neg_flat)                    # (B*K,)
            neg_score = neg_score.view(B, K)                            # (B, K)
            pos_score, _, _ = model(user_ids, pos_ids)                  # (B,)

            if neg_ids.ndim == 1:                    # K = 1
             loss = -torch.log(torch.sigmoid(pos_score - neg_score)).mean()
            else:                                    # K > 1
             loss = -torch.log(torch.sigmoid(pos_score.unsqueeze(1) - neg_score)).mean()

            # 5.反向 & 更新
            optimizer.zero_grad()
            loss.backward()
            if max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()

            epoch_loss += loss.item()

        # —— epoch 结束，打印日志 ——
        avg_loss = epoch_loss / len(train_data_loader)
        dt_end = datetime.now()
        dt = dt_end - dt_start

        print(f"[Epoch {epoch:02d}/{epochs}]  avg BPR Loss = {avg_loss:.4f},takes {dt.total_seconds()}s")

    return



In [12]:
model = DSSM(num_users,num_items,emb_dim=64)
model.to(device)
train_model(model=model,epochs=2, train_df=train_df,num_items=num_items,batch_size=512)

[Epoch 01/2]  avg BPR Loss = 0.6892,takes 90.085939s
[Epoch 02/2]  avg BPR Loss = 0.6789,takes 86.551551s


In [13]:
model.save_embeddings(num_users=num_users,num_items=num_items,device=device,save_dir=save_dir)

Saved user/item embeddings and FAISS index.


In [14]:
def hit_rate_at_k(ranked_list, true_item):
    return int(true_item in ranked_list)

In [15]:
def ndcg_at_k(ranked_list, true_item):
    if true_item in ranked_list:
        index = ranked_list.index(true_item)
        return 1 / np.log2(index + 2)
    else:
        return 0.0

In [16]:
def evaluate_random(test_loader, item_pool, top_k=10):
    hits, ndcgs = [], []
    for _, item_batch in test_loader:
        item_batch = item_batch.numpy()
        for true_item in item_batch:
            # 从全体物品中随机抽 top_k 个（包含或不包含 true_item）
            rec_list = random.sample(item_pool, top_k)
            hits.append(hit_rate_at_k(rec_list, true_item))
            ndcgs.append(ndcg_at_k(rec_list, true_item))
    return np.mean(hits), np.mean(ndcgs)


In [17]:

def evaluate_popular(test_loader, train_df, top_k=10):
    item_counts = Counter(train_df['item_id'].values)
    popular_items = [item for item, _ in item_counts.most_common(top_k)]

    hits, ndcgs = [], []
    for _, item_batch in test_loader:
        item_batch = item_batch.numpy()
        for true_item in item_batch:
            hits.append(hit_rate_at_k(popular_items, true_item))
            ndcgs.append(ndcg_at_k(popular_items, true_item))
    return np.mean(hits), np.mean(ndcgs)

In [18]:
def evaluate_model(test_loader, model, faiss_index, device, top_k=10):
    hits, ndcgs = [], []
    model.eval()

    for user_batch, item_batch in test_loader:
        user_batch = user_batch.to(device)
        item_batch = item_batch.cpu().numpy()  # true items

        with torch.no_grad():
            user_vecs = model.get_users_embedding(user_batch)
            user_vecs = user_vecs.cpu().numpy().astype(np.float32)

        # FAISS 批量 topK
        _, I = faiss_index.search(user_vecs, top_k)  # (B, K)
        topk_lists = I.tolist()

        for rec_list, true_item in zip(topk_lists, item_batch):
            hits.append(hit_rate_at_k(rec_list, true_item))
            ndcgs.append(ndcg_at_k(rec_list, true_item))

    return np.mean(hits), np.mean(ndcgs)

In [19]:
test_loader = customdataset.build_test_loader(test_df, num_items ,user_col = user_id, item_col = item_id, batch_size=64)
item_pool = list(range(num_items))
faiss_index = faiss.read_index(f"{save_dir}/item_index.faiss")

hr_r, ndcg_r = evaluate_random(test_loader, item_pool ,top_k=top_k)

print(f"Random HR@{top_k} = {hr_r:.4f}, nDCG@{top_k} = {ndcg_r:.4f}")


Random HR@10 = 0.0006, nDCG@10 = 0.0002


In [20]:
hr_p, ndcg_p = evaluate_popular(test_loader, train_df,top_k=top_k)
print(f"Popular HR@{top_k} = {hr_p:.4f}, nDCG@{top_k} = {ndcg_p:.4f}")


Popular HR@10 = 0.0029, nDCG@10 = 0.0014


In [21]:
hr_m, ndcg_m = evaluate_model(test_loader, model, faiss_index, device,top_k=top_k)
print(f"Model   HR@{top_k} = {hr_m:.4f}, nDCG@{top_k} = {ndcg_m:.4f}")


Model   HR@10 = 0.0017, nDCG@10 = 0.0007
